# Pipeline
1. Check for new epsisodes
2. Download WAV file
3. Transcribe & Diarize wav > csv
4. Name Speakers
5. Create film lists & summary
6. Selenium Upload

In [ ]:
!pip install langchain --quiet
!pip install langchain_openai --quiet
!pip install feedparser --quiet
!pip install git+https://github.com/m-bain/whisperx.git --upgrade --quiet
!pip install ctranslate2==4.4.0 --quiet

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/TBP_Project/python_functions')
sys.path.append('/content/drive/MyDrive/TBP_Project/podcast_downloader')

from speaker_naming import process_speaker01_file
from list_creation import create_movie_lists
import list_creation

import warnings
warnings.filterwarnings("ignore")
import pandas as pd

# Set max column width to display long strings
pd.set_option('display.max_colwidth', None)  # Set to None to prevent truncation

In [ ]:
from google.colab import userdata
OPEN_API_KEY = userdata.get('OPENAI_API_KEY')
LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')

In [ ]:
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['OPENAI_API_KEY'] = OPEN_API_KEY

In [ ]:
# Define the directory and load the RSS feed DataFrame
directory_path = "/content/drive/MyDrive/TBP_Project/"
transcripts_path = "/content/drive/MyDrive/TBP_Project/Transcripts/"
csv_path = "/content/drive/MyDrive/TBP_Project/CSV/"

In [ ]:
# Check for new episodes not in letterboxd
from file_upload_df_from_rss import create_upload_file_df, create_files_to_process
# files_to_process_df = create_upload_file_df(directory_path, '.txt')
files_to_process_df = create_upload_file_df(directory_path, '.txt')

# Count of files from podcast RSS feed (last 50 episodes) not currently in letterboxd
len(files_to_process_df)

## Download Audio with podcast_downloader

In [ ]:
# Download WAV files to Transcripts folder. Default behavior to download last 5 episodes.
!pip install podcast_downloader --quiet
%cd /content/drive/MyDrive/TBP_Project/podcast_downloader
!pip install podcast_downloader
!python -m podcast_downloader --config /content/drive/MyDrive/TBP_Project/podcast_downloader/podcast-downloader/the_big_picture_downloader_config.json

In [ ]:
# Return to main repo
%cd /content/drive/MyDrive/TBP_Project

## Diarization of Audio using Whisper to CSV

In [ ]:
# Transcribe new episodes w/ diarization as SPEAKER01 files
from transcription import process_diarization
process_diarization(transcripts_path)

## Name Speakers and convert csv to txt

In [ ]:
# Process speaker_01 csvs to create named .txt diarizations speaker file
process_speaker01_file(transcripts_path)

## Create Letterboxd List components in CSV

In [ ]:
# Create movie lists
create_movie_lists(transcripts_path)

## Upload list components to Letterboxd with Selenium

In [ ]:
# Upload files in CSV folder
# Note this requires GPU (T4 is fine) Chrome Driver fails on CPU
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install selenium --quiet
!pip install webdriver-manager --quiet
from selenium_list_upload import selenium_upload
selenium_upload(directory_path)

In [ ]:
print('Completed Scheduled Run')